In [ ]:
%load_ext tensorboard

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

MODEL_NAME = "ai-forever/ruT5-base"
MODEL_PATH = "./models/t5_model"

model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, legacy=False)

# Сохранение модели и токенизатора на диск
model.save_pretrained(MODEL_PATH)
tokenizer.save_pretrained(MODEL_PATH)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('ai-forever/ruT5-base\\tokenizer_config.json',
 'ai-forever/ruT5-base\\special_tokens_map.json',
 'ai-forever/ruT5-base\\spiece.model',
 'ai-forever/ruT5-base\\added_tokens.json')

In [ ]:
from py.misс import JSONDataLoader, extract_sbsq, extract_daNetQA, find_max_batch_size
from py.models.data import SquadQADataset, DaNetQADataset, CombinedDataModule
from transformers import T5Tokenizer

# Оптимальный BATCH_SIZE зависит от памяти GPU:
# - RTX 3090 (24GB VRAM):
#   - T5-small → 32–64
#   - T5-base → 16–32
#   - T5-large → 8–16
#   - T5-3B → 2–4
# Используйте `find_max_batch_size()` для точного подбора.
# Малый батч (≤16): более шумный градиент, но лучшее обобщение.
# Большой батч (≥64): более стабильный градиент, но может переобучаться.
# При нехватке памяти → уменьшите batch_size или включите gradient accumulation.
BATCH_SIZE = 8
N_EPOCHS = 4

# Загрузка токенизатора
special_tokens = {'additional_special_tokens': ['[КОНТЕКСТ]', '[ВОПРОС]']}
tokenizer: T5Tokenizer = T5Tokenizer.from_pretrained(MODEL_PATH, legacy=False)
tokenizer.add_special_tokens(special_tokens)

# Максимальная длина токенов
MAX_TEXT_TOKEN_LEN = 396 # Попробывать разные значения: 512, 1024 

json_data_loader = JSONDataLoader(tokenizer = tokenizer, max_text_token_len = MAX_TEXT_TOKEN_LEN)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# Загрузка тренировочного набора данных sberquad
squad_df = json_data_loader.load_from_file(extraction_function = extract_sbsq, file_path="datasets\\sberquad\\train_v1.0\\train_v1.0.json")
squad_dataset = SquadQADataset(squad_df, tokenizer=tokenizer, text_max_token_len=MAX_TEXT_TOKEN_LEN, summary_max_token_len=1024)
# Загрузка валидационного набора данных sberquad
val_squad_df = json_data_loader.load_from_file(extraction_function = extract_sbsq, file_path="datasets\\sberquad\\dev_v1.0\\dev_v1.0.json")
val_squad_dataset = SquadQADataset(val_squad_df, tokenizer=tokenizer, text_max_token_len=MAX_TEXT_TOKEN_LEN, summary_max_token_len=1024)

KeyboardInterrupt: 

In [3]:
# Загрузка тренировочного набора данных sberquad DaNetQA
daNetQA_df = json_data_loader.load_from_file(extraction_function = extract_daNetQA, file_path="datasets\\RussianSuperGlue\\DaNetQA\\train.jsonl")
daNetQA_dataset = DaNetQADataset(daNetQA_df, tokenizer=tokenizer, text_max_token_len=MAX_TEXT_TOKEN_LEN, summary_max_token_len=1024)
# Загрузка валидационного набора данных DaNetQA
val_daNetQA_df = json_data_loader.load_from_file(extraction_function = extract_daNetQA, file_path="datasets\\RussianSuperGlue\\DaNetQA\\val.jsonl")
val_daNetQA_dataset = SquadQADataset(val_daNetQA_df, tokenizer=tokenizer, text_max_token_len=MAX_TEXT_TOKEN_LEN, summary_max_token_len=1024)

In [4]:
# Инициализация DataModule
train_dataloader = CombinedDataModule(
    tokenizer=tokenizer,
    train_datasets=[squad_dataset, daNetQA_dataset],
    val_datasets=[val_squad_dataset, val_daNetQA_dataset],
    train_batch_size=BATCH_SIZE,
    eval_batch_size=BATCH_SIZE,
    num_workers=4
)

# Настройка DataLoader
train_dataloader.setup(stage='fit')

In [15]:
train_dataloader.train_weights

[0.2735176086425781, 0.7264823317527771]

In [16]:
[len(squad_dataset.data), len(daNetQA_dataset.data)]

[44893, 1730]

In [ ]:
from py.models.model import QAModule
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import (ModelCheckpoint, EarlyStopping)
from transformers import T5ForConditionalGeneration

import torch
#pl.seed_everything(42, workers=True)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
t5_model = T5ForConditionalGeneration.from_pretrained(MODEL_PATH, return_dict=True)
# t5_model.gradient_checkpointing_enable() #Это уменьшает потребление памяти за счет дополнительного вычисления.
model = QAModule(t5_model, lr=1e-4, weight_decay=1e-5)
model.to(device)

torch.set_float32_matmul_precision('high')

tb_logger = TensorBoardLogger('logs', name="t5_qa",)

checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath="checkpoints",
    filename="best_model-{epoch:02d}-{val_loss:.2f}",
    save_top_k=1,
    mode="min",
    every_n_epochs=1,
    verbose=True
)

early_stopping_callback = EarlyStopping(
    monitor="val_loss",
    patience=3,  # Количество эпох без улучшения до остановки
    mode="min",
    verbose=True
)

trainer = pl.Trainer(
    callbacks=[checkpoint_callback, early_stopping_callback],
    logger=tb_logger,
    max_epochs=N_EPOCHS,
    accelerator="gpu",
    devices=1,
    log_every_n_steps=1,
    accumulate_grad_batches=8, #накапливать каждые 8 батча (эффективный batch size — batch_size*8) # Чем больше число - тем меньше число шагов обратного распространения.    
    precision="16-mixed" # Используем смешанную точность
)



Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [18]:
torch.cuda.memory_summary()

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   4275 MiB |  30680 MiB | 199517 GiB | 199513 GiB |\n|       from large pool |   4274 MiB |  30677 MiB | 199485 GiB | 199481 GiB |\n|       from small pool |      1 MiB |      3 MiB |     32 GiB |     32 GiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   4275 MiB |  30680 MiB | 199517 GiB | 199513 GiB |\n|       from large pool |   4274 MiB |  30677 MiB |

In [6]:
trainer.fit(model, train_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params | Mode
------------------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M  | eval
------------------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

c:\Users\SawKing\Documents\T5\.venv\Lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
trainer.validate(model, train_dataloader)

In [ ]:
trainer.test(datamodule=train_dataloader)